In [291]:
from IPython.display import display, Math
import sympy as sp
from sympy import sin, cos
import os
import pickle

## Lagrange Calculator

### Define Symbols

In [292]:
# Time symbol
t = sp.symbols('t')

# Define functions of time
x = sp.Function('x')(t)
theta = sp.Function('theta')(t)

# First derivatives
dx = sp.diff(x, t)
dtheta = sp.diff(theta, t)

# Second derivatives
ddx = sp.diff(dx, t)
ddtheta = sp.diff(dtheta, t)

In [293]:
# Parameters
r, Mb, rb, Ib, Mw, Iw, g, tau = sp.symbols('r M_b r_b I_b M_w I_w g tau')

#### Define a Latex Converter For Better Readability

In [294]:
def Latex_Dmath_Converter(expr, header, collect_flag=True):
    # Define symbols and functions
    t, r = sp.symbols('t r')
    x_func = sp.Function('x')(t)
    theta_func = sp.Function('theta')(t)

    # Define derivatives
    dx = sp.diff(x_func, t)
    ddx = sp.diff(x_func, t, 2)
    dtheta = sp.diff(theta_func, t)
    ddtheta = sp.diff(theta_func, t, 2)

    # LaTeX symbol mapping
    subs_dict = {
        x_func: sp.Symbol('x'),
        theta_func: sp.Symbol(r'\theta'),
        dx: sp.Symbol(r'\dot{x}'),
        ddx: sp.Symbol(r'\ddot{x}'),
        dtheta: sp.Symbol(r'\dot{\theta}'),
        ddtheta: sp.Symbol(r'\ddot{\theta}'),
    }

    # Apply substitution
    expr_sub = expr.subs(subs_dict)

    # Check if expr is an equation (Eq)
    if isinstance(expr_sub, sp.Equality):
        lhs, rhs = expr_sub.lhs, expr_sub.rhs

        # Optionally collect terms on lhs and rhs
        if collect_flag:
            collect_symbols = list(subs_dict.values())
            lhs_collected = sp.collect(lhs, collect_symbols)
            rhs_collected = sp.collect(rhs, collect_symbols)

            lhs_latex = sp.latex(lhs_collected)
            rhs_latex = sp.latex(rhs_collected)

        else:
            lhs_latex = sp.latex(lhs)
            rhs_latex = sp.latex(rhs)

        latex_str = f"{lhs_latex} = {rhs_latex}"

    else:
        # If not an equation
        if collect_flag:
            collect_symbols = list(subs_dict.values())
            expr_sub_collect = sp.collect(expr_sub, collect_symbols)
            latex_str = sp.latex(expr_sub_collect)
        else:
            latex_str = sp.latex(expr_sub)


    # Output
    print(r"\begin{multline*}")
    print(header, latex_str)
    print(r"\end{multline*}")

    return(f"{header}{latex_str}")


### Kinematic and Potential Terms

In [295]:
T1 = (1/2)*Mw*(dx**2) # Wheel Translation K
T2 = (1/2)*Iw*((dx/r)**2) # Wheel Rotatory K
T3 = (1/2)*Mb*((dx**2)+(2*dx*(dtheta)*rb*cos(theta))+((dtheta)**2)*(rb**2)) # Body Translation K
T4 = (1/2)*Ib*((dtheta)**2)# Wheel Rotatory K
T = T1+T2+T3+T4

In [296]:
V = Mb*g*rb*cos(theta)

In [297]:
L = T-V

In [298]:
T_expr = Latex_Dmath_Converter(T,r"T=",False)
V_expr =Latex_Dmath_Converter(V,r"V=",False)
L_expr =Latex_Dmath_Converter(L,r"L=",False)

\begin{multline*}
T= 0.5 I_{b} \dot{\theta}^{2} + \frac{0.5 I_{w} \dot{x}^{2}}{r^{2}} + 0.5 M_{b} \left(\dot{\theta}^{2} r_{b}^{2} + 2 \dot{\theta} \dot{x} r_{b} \cos{\left(\theta \right)} + \dot{x}^{2}\right) + 0.5 M_{w} \dot{x}^{2}
\end{multline*}
\begin{multline*}
V= M_{b} g r_{b} \cos{\left(\theta \right)}
\end{multline*}
\begin{multline*}
L= 0.5 I_{b} \dot{\theta}^{2} + \frac{0.5 I_{w} \dot{x}^{2}}{r^{2}} - M_{b} g r_{b} \cos{\left(\theta \right)} + 0.5 M_{b} \left(\dot{\theta}^{2} r_{b}^{2} + 2 \dot{\theta} \dot{x} r_{b} \cos{\left(\theta \right)} + \dot{x}^{2}\right) + 0.5 M_{w} \dot{x}^{2}
\end{multline*}


In [299]:
display(Math(T_expr))
display(Math(V_expr))
display(Math(L_expr))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Generalized Coordinates Functions

In [300]:
def coordinate_x(L, RHS):
    dL_ddx = sp.diff(L, dx)
    d_dt_dL_ddx = sp.diff(dL_ddx, t)
    dL_dx = sp.diff(L, x)
    
    lagrange_eq = sp.Eq(d_dt_dL_ddx - dL_dx, RHS)
    return dL_ddx, d_dt_dL_ddx, dL_dx, lagrange_eq


def coordinate_theta(L, RHS):
    dL_ddtheta = sp.diff(L, dtheta)
    d_dt_dL_ddtheta = sp.diff(dL_ddtheta, t)
    dL_dtheta = sp.diff(L, theta)

    lagrange_eq = sp.Eq(d_dt_dL_ddtheta - dL_dtheta, RHS)
    return dL_ddtheta, d_dt_dL_ddtheta, dL_dtheta, lagrange_eq

###  Calculate the diff

In [301]:
dL_ddx, d_dt_dL_ddx, dL_dx, lagrange_eq_x = coordinate_x(L, -tau/r)
dL_ddtheta, d_dt_dL_ddtheta, dL_dtheta, lagrange_eq_theta = coordinate_theta(L, tau)

In [302]:
print(dL_dtheta)
print(lagrange_eq_x)
print(lagrange_eq_theta)

M_b*g*r_b*sin(theta(t)) - 1.0*M_b*r_b*sin(theta(t))*Derivative(theta(t), t)*Derivative(x(t), t)
Eq(1.0*I_w*Derivative(x(t), (t, 2))/r**2 + 0.5*M_b*(-2*r_b*sin(theta(t))*Derivative(theta(t), t)**2 + 2*r_b*cos(theta(t))*Derivative(theta(t), (t, 2)) + 2*Derivative(x(t), (t, 2))) + 1.0*M_w*Derivative(x(t), (t, 2)), -tau/r)
Eq(1.0*I_b*Derivative(theta(t), (t, 2)) - M_b*g*r_b*sin(theta(t)) + 1.0*M_b*r_b*sin(theta(t))*Derivative(theta(t), t)*Derivative(x(t), t) + 0.5*M_b*(2*r_b**2*Derivative(theta(t), (t, 2)) - 2*r_b*sin(theta(t))*Derivative(theta(t), t)*Derivative(x(t), t) + 2*r_b*cos(theta(t))*Derivative(x(t), (t, 2))), tau)


#### Print For Latex

In [303]:
dL_ddx_expr = Latex_Dmath_Converter(dL_ddx, r"\frac{\partial L}{\partial \dot{x}}=")
d_dt_dL_ddx_expr = Latex_Dmath_Converter(d_dt_dL_ddx, r"\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{x}} \right) =")
dL_dx_expr = Latex_Dmath_Converter(dL_dx, r"\frac{\partial L}{\partial x}=")
lagrange_eq_x_expr = Latex_Dmath_Converter(lagrange_eq_x, r"\text{Coordinate X: }")

dL_ddtheta_expr = Latex_Dmath_Converter(dL_ddtheta, r"\frac{\partial L}{\partial \dot{\theta}}=")
d_dt_dL_ddtheta_expr = Latex_Dmath_Converter(d_dt_dL_ddtheta, r"\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{\theta}} \right) =")
dL_dtheta_expr = Latex_Dmath_Converter(dL_dtheta, r"\frac{\partial L}{\partial \theta}=")
lagrange_eq_theta_expr = Latex_Dmath_Converter(lagrange_eq_theta, r"\text{Coordinate } \theta: ")



\begin{multline*}
\frac{\partial L}{\partial \dot{x}}= 0.5 M_{b} \left(2 \dot{\theta} r_{b} \cos{\left(\theta \right)} + 2 \dot{x}\right) + \dot{x} \left(\frac{1.0 I_{w}}{r^{2}} + 1.0 M_{w}\right)
\end{multline*}
\begin{multline*}
\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{x}} \right) = 0.5 M_{b} \left(2 \ddot{\theta} r_{b} \cos{\left(\theta \right)} + 2 \ddot{x} - 2 \dot{\theta}^{2} r_{b} \sin{\left(\theta \right)}\right) + \ddot{x} \left(\frac{1.0 I_{w}}{r^{2}} + 1.0 M_{w}\right)
\end{multline*}
\begin{multline*}
\frac{\partial L}{\partial x}= 0
\end{multline*}
\begin{multline*}
\text{Coordinate X: } 0.5 M_{b} \left(2 \ddot{\theta} r_{b} \cos{\left(\theta \right)} + 2 \ddot{x} - 2 \dot{\theta}^{2} r_{b} \sin{\left(\theta \right)}\right) + \ddot{x} \left(\frac{1.0 I_{w}}{r^{2}} + 1.0 M_{w}\right) = - \frac{\tau}{r}
\end{multline*}
\begin{multline*}
\frac{\partial L}{\partial \dot{\theta}}= 1.0 I_{b} \dot{\theta} + 0.5 M_{b} \left(2 \dot{\theta} r_{b}^{2} + 2 \dot{x} r_{b} \co

In [304]:
display(Math(lagrange_eq_x_expr))
display(Math(lagrange_eq_theta_expr))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Matrix Form

In [305]:
eqs = [lagrange_eq_x.lhs-lagrange_eq_x.rhs, lagrange_eq_theta.lhs-lagrange_eq_theta.rhs]
vars = [ddx, ddtheta]
A, b = sp.linear_eq_to_matrix(eqs, vars)

In [306]:
display(A)
display(b)

Matrix([
[1.0*I_w/r**2 + 1.0*M_b + 1.0*M_w, 1.0*M_b*r_b*cos(theta(t))],
[       1.0*M_b*r_b*cos(theta(t)),  1.0*I_b + 1.0*M_b*r_b**2]])

Matrix([
[1.0*M_b*r_b*sin(theta(t))*Derivative(theta(t), t)**2 - tau/r],
[                               M_b*g*r_b*sin(theta(t)) + tau]])

### Solve the Equation

In [307]:
solutions = sp.solve([lagrange_eq_x, lagrange_eq_theta], (ddx, ddtheta))
xdd_solution = solutions[ddx]
thetadd_solution = solutions[ddtheta]

In [308]:
xdd_solution_expr = Latex_Dmath_Converter(xdd_solution,r"\ddot{x}=", collect_flag=False)
thetadd_solution_expr = Latex_Dmath_Converter(thetadd_solution,r"\ddot{\theta}=", collect_flag= False)

\begin{multline*}
\ddot{x}= \frac{I_{b} M_{b} \dot{\theta}^{2} r^{2} r_{b} \sin{\left(\theta \right)}}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} M_{b} r_{b}^{2} - M_{b}^{2} r^{2} r_{b}^{2} \cos^{2}{\left(\theta \right)} + M_{b}^{2} r^{2} r_{b}^{2} + M_{b} M_{w} r^{2} r_{b}^{2}} - \frac{I_{b} r \tau}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} M_{b} r_{b}^{2} - M_{b}^{2} r^{2} r_{b}^{2} \cos^{2}{\left(\theta \right)} + M_{b}^{2} r^{2} r_{b}^{2} + M_{b} M_{w} r^{2} r_{b}^{2}} + \frac{M_{b}^{2} \dot{\theta}^{2} r^{2} r_{b}^{3} \sin{\left(\theta \right)}}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} M_{b} r_{b}^{2} - M_{b}^{2} r^{2} r_{b}^{2} \cos^{2}{\left(\theta \right)} + M_{b}^{2} r^{2} r_{b}^{2} + M_{b} M_{w} r^{2} r_{b}^{2}} - \frac{M_{b}^{2} g r^{2} r_{b}^{2} \sin{\left(\theta \right)} \cos{\left(\theta \right)}}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} M_{b} r_{b}^{2} - M_{b}^{2} r^{2} r_{b}^{2} \cos^{2}{\left(\the

In [309]:
display(Math(xdd_solution_expr))
display(Math(thetadd_solution_expr))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Matrix Solution

In [310]:
solution_matrix = A.LUsolve(b)
print(solution_matrix)
xdd_matrix_solution = solution_matrix[0, 0]
thetadd_matrix_solution = solution_matrix[1, 0]

Matrix([[(1.0*M_b*r_b*sin(theta(t))*Derivative(theta(t), t)**2 - 1.0*M_b*r_b*(M_b*g*r_b*sin(theta(t)) - 1.0*M_b*r_b*(1.0*M_b*r_b*sin(theta(t))*Derivative(theta(t), t)**2 - tau/r)*cos(theta(t))/(1.0*I_w/r**2 + 1.0*M_b + 1.0*M_w) + tau)*cos(theta(t))/(1.0*I_b - 1.0*M_b**2*r_b**2*cos(theta(t))**2/(1.0*I_w/r**2 + 1.0*M_b + 1.0*M_w) + 1.0*M_b*r_b**2) - tau/r)/(1.0*I_w/r**2 + 1.0*M_b + 1.0*M_w)], [(M_b*g*r_b*sin(theta(t)) - 1.0*M_b*r_b*(1.0*M_b*r_b*sin(theta(t))*Derivative(theta(t), t)**2 - tau/r)*cos(theta(t))/(1.0*I_w/r**2 + 1.0*M_b + 1.0*M_w) + tau)/(1.0*I_b - 1.0*M_b**2*r_b**2*cos(theta(t))**2/(1.0*I_w/r**2 + 1.0*M_b + 1.0*M_w) + 1.0*M_b*r_b**2)]])


In [311]:
xdd_matrix_solution_expr = Latex_Dmath_Converter(xdd_matrix_solution,r"\ddot{x}=", collect_flag=False)
thetadd_matrix_solution_expr = Latex_Dmath_Converter(thetadd_matrix_solution,r"\ddot{\theta}=", collect_flag= False)

display(Math(xdd_matrix_solution_expr))
display(Math(thetadd_matrix_solution_expr))

\begin{multline*}
\ddot{x}= \frac{1.0 M_{b} \dot{\theta}^{2} r_{b} \sin{\left(\theta \right)} - \frac{1.0 M_{b} r_{b} \left(M_{b} g r_{b} \sin{\left(\theta \right)} - \frac{1.0 M_{b} r_{b} \left(1.0 M_{b} \dot{\theta}^{2} r_{b} \sin{\left(\theta \right)} - \frac{\tau}{r}\right) \cos{\left(\theta \right)}}{\frac{1.0 I_{w}}{r^{2}} + 1.0 M_{b} + 1.0 M_{w}} + \tau\right) \cos{\left(\theta \right)}}{1.0 I_{b} - \frac{1.0 M_{b}^{2} r_{b}^{2} \cos^{2}{\left(\theta \right)}}{\frac{1.0 I_{w}}{r^{2}} + 1.0 M_{b} + 1.0 M_{w}} + 1.0 M_{b} r_{b}^{2}} - \frac{\tau}{r}}{\frac{1.0 I_{w}}{r^{2}} + 1.0 M_{b} + 1.0 M_{w}}
\end{multline*}
\begin{multline*}
\ddot{\theta}= \frac{M_{b} g r_{b} \sin{\left(\theta \right)} - \frac{1.0 M_{b} r_{b} \left(1.0 M_{b} \dot{\theta}^{2} r_{b} \sin{\left(\theta \right)} - \frac{\tau}{r}\right) \cos{\left(\theta \right)}}{\frac{1.0 I_{w}}{r^{2}} + 1.0 M_{b} + 1.0 M_{w}} + \tau}{1.0 I_{b} - \frac{1.0 M_{b}^{2} r_{b}^{2} \cos^{2}{\left(\theta \right)}}{\frac{1.0 I_{w}}{r^{

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [312]:
sp.simplify(xdd_solution - xdd_matrix_solution)  
sp.simplify(thetadd_solution - thetadd_matrix_solution) 

0

matched!

### Linearization

In [313]:
#Define constant tau0
tau0 = sp.symbols('tau0')

# State vector
X = sp.Matrix([x, dx, theta, dtheta])

# Dynamics vector
dX = sp.Matrix([dx, xdd_solution, dtheta, thetadd_solution])

# Equilibrium point
eq_point = {x: 0, dx: 0, theta: 0, dtheta: 0, tau: tau0}

# Compute Jacobians and evaluate at equilibrium
A = dX.jacobian(X).subs(eq_point)
B = dX.jacobian([tau]).subs(eq_point)

# Print A matrix in LaTeX
print("A matrix in LaTeX:")
print(sp.latex(A))

# Print B matrix in LaTeX
print("\nB matrix in LaTeX:")
print(sp.latex(B))

A matrix in LaTeX:
\left[\begin{matrix}0 & 1 & 0 & 0\\0 & 0 & - \frac{M_{b}^{2} g r^{2} r_{b}^{2}}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} M_{b} r_{b}^{2} + M_{b} M_{w} r^{2} r_{b}^{2}} & 0\\0 & 0 & 0 & 1\\0 & 0 & \frac{I_{w} M_{b} g r_{b}}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} M_{b} r_{b}^{2} + M_{b} M_{w} r^{2} r_{b}^{2}} + \frac{M_{b}^{2} g r^{2} r_{b}}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} M_{b} r_{b}^{2} + M_{b} M_{w} r^{2} r_{b}^{2}} + \frac{M_{b} M_{w} g r^{2} r_{b}}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} M_{b} r_{b}^{2} + M_{b} M_{w} r^{2} r_{b}^{2}} & 0\end{matrix}\right]

B matrix in LaTeX:
\left[\begin{matrix}0\\- \frac{I_{b} r}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} M_{b} r_{b}^{2} + M_{b} M_{w} r^{2} r_{b}^{2}} - \frac{M_{b} r^{2} r_{b}}{I_{b} I_{w} + I_{b} M_{b} r^{2} + I_{b} M_{w} r^{2} + I_{w} M_{b} r_{b}^{2} + M_{b} M_{w} r^{2} r_{b}^{2}} - \frac{M_{b} r r_{b}^{2}}

### Output to a file

In [314]:
os.makedirs('output', exist_ok=True)
with open('output/xdd_solution.txt', 'w') as f:
    f.write(str(xdd_solution))

with open('output/thetadd_solution.txt', 'w') as f:
    f.write(str(thetadd_solution))

with open('output/A.txt', 'w') as f:
    f.write(str(A))

with open('output/B.txt', 'w') as f:
    f.write(str(B))